In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
from time import sleep
from datetime import datetime
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service

In [2]:
def scrape_page_selenium(link):
    driver_path = "F:\geckodriver.exe"
    service = Service(driver_path)
    driver = webdriver.Firefox(service=service)
    try:
        driver.get(link)
        driver.implicitly_wait(3)
        html = driver.page_source
        response = requests.get(link)
        if response.status_code ==200:
            soup = BeautifulSoup(html)
            try:
                brand = soup.find('h1', {'class': 'vehicle-detail-title mb-0'}).text.strip().split()[0]
                
            except:
                brand = np.nan
            try:
                model = soup.find('h1', {'class': 'vehicle-detail-title mb-0'}).text.strip().split()[1]
                
            except:
                model = np.nan
            try:
                price = soup.find('div', {'class': 'price-wrapper d-flex'}).find_all('h3', {'class': 'price-number mb-0'})[0].text.strip().replace('€','')
                
            except:
                price = np.nan
                print('Not found')
                
            try:
                displacement = soup.find_all('span', {'class': 'vehicle-information-item-text d-block font-weight-bold'})[0].text.strip().split()[0]
            except:
                displacement = np.nan
                
            try:
                horsepower = soup.find_all('span', {'class': 'vehicle-information-item-text d-block font-weight-bold'})[1].text.strip().split()[0]
            except:
                horsepower = np.nan
                
            try:
                gearbox = soup.find_all('span', {'class': 'vehicle-information-item-text d-block font-weight-bold'})[2].text.strip()
            except:
                gearbox = np.nan
                
            try:
                fuel = soup.find_all('span', {'class': 'vehicle-information-item-text d-block font-weight-bold'})[4].text.strip()
            except:
                fuel = np.nan
                
            try:
                consumption = soup.find_all('span', {'class': 'vehicle-information-item-text d-block font-weight-bold'})[5].text.strip().split()[0]
            except:
                consumption = np.nan
                
                
            scrape_data = {
                'date': datetime.now(),
                'link': link,
                'brand': brand,
                'model': model,
                'price': price,
                'displacement': displacement,
                'horsepower': horsepower,
                'gearbox': gearbox,
                'fuel': fuel,
                'consumption': consumption      
            }
        
            return scrape_data
        else:
            print('server refused')
    finally:
        driver.quit()

In [3]:
#testing
scrape_page_selenium('https://www.instacar.gr/leasing/fiat/500?skuId=IC00243')

{'date': datetime.datetime(2024, 1, 23, 22, 17, 53, 11161),
 'link': 'https://www.instacar.gr/leasing/fiat/500?skuId=IC00243',
 'brand': 'FIAT',
 'model': '500',
 'price': '340',
 'displacement': '1242',
 'horsepower': '69',
 'gearbox': 'Χειροκίνητο',
 'fuel': 'Βενζίνη',
 'consumption': '5.7'}

### Get links for each car card

In [4]:
url = "https://www.instacar.gr/leasing/metaxeirismena"

response = requests.get(url,timeout=15)
response.status_code

200

In [5]:
soup = BeautifulSoup(response.content, "html.parser")

In [6]:
boxes = soup.find_all('div', {'class': 'leasing-list-item'})

In [7]:
len(boxes)

436

In [8]:
boxes[0]

<div class="leasing-list-item"><a class="vehicle-preview-card" href="/leasing/citroen/ami?skuId=IC00018"><div class="vehicle-preview-card-image"><img alt="CITROEN AMI" height="223" loading="eager" src="https://www.instacar.gr/cdn-cgi/image/format=auto,metadata=none,width=600,height=446,fit=pad,background=transparent/https%3A%2F%2Fd10ax2wcinhk08.cloudfront.net%2Fvehicles-catalog%2Ff4b469ff-fc91-402f-9cfc-69f0a81d61e8_citroen-ami.png" width="300"/> <div class="me-to-dexi-card-image"><img alt="Ευέλικτο leasing αυτοκινήτων" loading="lazy" src="/_nuxt/img/me-to-dexi-media-el.9c7b683.png"/></div> <div class="vehicles-cta-ribbon-wrapper"><div class="vehicles-cta-ribbon vehicles-cta-ribbon-ready-deliver text-uppercase">
    Ετοιμοπαραδοτο
  </div></div></div> <div class="vehicle-preview-card-content"><div><h3 class="vehicle-name font-weight-bold">
        CITROEN AMI
        <br/> <span class="vehicle-edition">
</span></h3></div> <div class="d-flex justify-content-between align-items-end"><div

In [9]:
boxes[0].find('a', {'class': 'vehicle-preview-card'}).get('href')

'/leasing/citroen/ami?skuId=IC00018'

In [10]:
main_link = 'https://www.instacar.gr'

car_url_list = []

for box in boxes:
    try:
        extend = box.find('a', {'class': 'vehicle-preview-card'}).get('href')
        page = main_link+extend
        car_url_list.append(page)
    except:
        print("Something went wrong")

Something went wrong
Something went wrong


In [11]:
len(car_url_list)

434

In [12]:
df = pd.DataFrame()

In [13]:
#testing

for link in car_url_list:
    print(link)

https://www.instacar.gr/leasing/citroen/ami?skuId=IC00018
https://www.instacar.gr/leasing/carver/cargo?skuId=IC00411
https://www.instacar.gr/leasing/seat/mii?skuId=IC00096
https://www.instacar.gr/leasing/skoda/citigo?skuId=IC00097
https://www.instacar.gr/leasing/fiat/panda?skuId=IC00050
https://www.instacar.gr/leasing/peugeot/208?skuId=IC00180
https://www.instacar.gr/leasing/xev/yoyo?skuId=IC00451
https://www.instacar.gr/leasing/fiat/500?skuId=IC00221
https://www.instacar.gr/leasing/fiat/500?skuId=IC00243
https://www.instacar.gr/leasing/kia/stonic?skuId=IC00214
https://www.instacar.gr/leasing/citroen/c3?skuId=IC00043
https://www.instacar.gr/leasing/seat/ibiza?skuId=IC00067
https://www.instacar.gr/leasing/citroen/c3?skuId=IC00357
https://www.instacar.gr/leasing/seat/arona?skuId=IC00086
https://www.instacar.gr/leasing/seat/ibiza?skuId=IC00167
https://www.instacar.gr/leasing/toyota/aygo-x?skuId=IC00401
https://www.instacar.gr/leasing/renault/clio?skuId=IC00098
https://www.instacar.gr/leas

In [14]:
for link in car_url_list:
    page_dict = scrape_page_selenium(link)
    page_df = pd.DataFrame([page_dict])
    df = pd.concat([df, page_df], axis = 0, ignore_index=True)

NoSuchWindowException: Message: Browsing context has been discarded
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:189:5
NoSuchWindowError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:630:5
assert.that/<@chrome://remote/content/shared/webdriver/Assert.sys.mjs:485:13
assert.open@chrome://remote/content/shared/webdriver/Assert.sys.mjs:147:4
GeckoDriver.prototype.navigateTo@chrome://remote/content/marionette/driver.sys.mjs:842:39
despatch@chrome://remote/content/marionette/server.sys.mjs:318:40
execute@chrome://remote/content/marionette/server.sys.mjs:289:16
onPacket/<@chrome://remote/content/marionette/server.sys.mjs:262:20
onPacket@chrome://remote/content/marionette/server.sys.mjs:263:9
_onJSONObjectReady/<@chrome://remote/content/marionette/transport.sys.mjs:494:20


In [15]:
df

,date,link,brand,model,price,displacement,horsepower,gearbox,fuel,consumption
0,2024-01-23 22:18:20.080421,https://www.instacar.gr/leasing/citroen/ami?sk...,CITROEN,AMI,230,5.5,10,Αυτόματο,Ηλεκτρικό,75
1,2024-01-23 22:18:40.544492,https://www.instacar.gr/leasing/carver/cargo?s...,CARVER,CARGO,273,5.4,10,Αυτόματο,Ηλεκτρικό,130
2,2024-01-23 22:18:59.857525,https://www.instacar.gr/leasing/seat/mii?skuId...,SEAT,Mii,285,999,60,Χειροκίνητο,Βενζίνη,4.5
3,2024-01-23 22:19:20.164083,https://www.instacar.gr/leasing/skoda/citigo?s...,SKODA,CITIGO,285,999,60,Χειροκίνητο,Βενζίνη,4.1
4,2024-01-23 22:19:39.028297,https://www.instacar.gr/leasing/fiat/panda?sku...,FIAT,PANDA,290,1242,69,Χειροκίνητο,Βενζίνη,5.3
...,...,...,...,...,...,...,...,...,...,...
222,2024-01-23 23:46:09.431678,https://www.instacar.gr/leasing/seat/ibiza?sku...,SEAT,IBIZA,346,999,75,Χειροκίνητο,Βενζίνη,4.9
223,2024-01-23 23:46:27.461917,https://www.instacar.gr/leasing/hyundai/i30?sk...,HYUNDAI,i30,348,998,120,Χειροκίνητο,Βενζίνη,5.5
224,2024-01-23 23:46:45.596435,https://www.instacar.gr/leasing/seat/ibiza?sku...,SEAT,IBIZA,350,999,90,Χειροκίνητο,Φυσικό αέριο,3.5
225,2024-01-23 23:47:04.762857,https://www.instacar.gr/leasing/seat/ibiza?sku...,SEAT,IBIZA,350,999,90,Χειροκίνητο,Φυσικό αέριο,3.5


In [25]:
car_url_list[226]

'https://www.instacar.gr/leasing/kia/stonic?skuId=IC00128'

In [26]:
second_list = car_url_list[227:]

In [27]:
second_list

['https://www.instacar.gr/leasing/peugeot/308?skuId=IC00308',
 'https://www.instacar.gr/leasing/peugeot/308?skuId=IC00309',
 'https://www.instacar.gr/leasing/hyundai/i30?skuId=IC00027',
 'https://www.instacar.gr/leasing/renault/kadjar?skuId=IC00378',
 'https://www.instacar.gr/leasing/seat/ibiza?skuId=IC00476',
 'https://www.instacar.gr/leasing/volkswagen/polo?skuId=IC00057',
 'https://www.instacar.gr/leasing/nissan/micra?skuId=IC00377',
 'https://www.instacar.gr/leasing/hyundai/i30?skuId=IC00125',
 'https://www.instacar.gr/leasing/seat/arona?skuId=IC00084',
 'https://www.instacar.gr/leasing/mg/zs?skuId=IC00442',
 'https://www.instacar.gr/leasing/nissan/juke?skuId=IC00362',
 'https://www.instacar.gr/leasing/hyundai/bayon?skuId=IC00123',
 'https://www.instacar.gr/leasing/mini/cooper?skuId=IC00303',
 'https://www.instacar.gr/leasing/volkswagen/polo?skuId=IC00114',
 'https://www.instacar.gr/leasing/skoda/citigo?skuId=IC00254',
 'https://www.instacar.gr/leasing/hyundai/kona?skuId=IC00073',


In [28]:
df2 = pd.DataFrame()

In [29]:
for link in second_list:
    page_dict = scrape_page_selenium(link)
    page_df = pd.DataFrame([page_dict])
    df2 = pd.concat([df2, page_df], axis = 0, ignore_index=True)

TimeoutException: Message: Navigation timed out after 300000 ms
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:189:5
TimeoutError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:691:5
bail@chrome://remote/content/marionette/sync.sys.mjs:211:19


In [30]:
df2

,date,link,brand,model,price,displacement,horsepower,gearbox,fuel,consumption
0,2024-01-23 23:53:12.685669,https://www.instacar.gr/leasing/peugeot/308?sk...,PEUGEOT,308,350,1560,99,Χειροκίνητο,Πετρέλαιο,5
1,2024-01-23 23:53:38.170266,https://www.instacar.gr/leasing/peugeot/308?sk...,PEUGEOT,308,350,1200,130,Χειροκίνητο,Βενζίνη,3.6
2,2024-01-23 23:53:57.158556,https://www.instacar.gr/leasing/hyundai/i30?sk...,HYUNDAI,i30,354,998,120,Χειροκίνητο,Βενζίνη,5.4
3,2024-01-23 23:54:19.827294,https://www.instacar.gr/leasing/renault/kadjar...,RENAULT,KADJAR,354,1461,115,Χειροκίνητο,Πετρέλαιο,4.6
4,2024-01-23 23:54:38.794641,https://www.instacar.gr/leasing/seat/ibiza?sku...,SEAT,IBIZA,354,1590,80,Χειροκίνητο,Πετρέλαιο,4
...,...,...,...,...,...,...,...,...,...,...
161,2024-01-24 00:58:19.412556,https://www.instacar.gr/leasing/mercedes/gla?s...,MERCEDES,GLA,992,1332,163,Αυτόματο,Βενζίνη,7.3
162,2024-01-24 00:58:43.568194,https://www.instacar.gr/leasing/toyota/proace-...,TOYOTA,PROACE,993,1998,144,Χειροκίνητο,Πετρέλαιο,6.4
163,2024-01-24 00:59:07.982562,https://www.instacar.gr/leasing/mercedes/glc?s...,MERCEDES,GLC,999,1991,320,Αυτόματο,Βενζίνη,2.2
164,2024-01-24 00:59:32.528091,https://www.instacar.gr/leasing/volkswagen/id-...,VOLKSWAGEN,ID.4,1000,77,204,Αυτόματο,Ηλεκτρικό,520


In [41]:
car_url_list[392]

'https://www.instacar.gr/leasing/mercedes/cla?skuId=IC00391'

In [48]:
third_list = car_url_list[393:]

In [37]:
df3 = pd.DataFrame()

In [38]:
for link in third_list:
    page_dict = scrape_page_selenium(link)
    page_df = pd.DataFrame([page_dict])
    df3 = pd.concat([df3, page_df], axis = 0, ignore_index=True)

In [39]:
df3

,date,link,brand,model,price,displacement,horsepower,gearbox,fuel,consumption
0,2024-01-24 01:10:31.450924,https://www.instacar.gr/leasing/tesla/model-3?...,TESLA,MODEL,1012,60,325,Αυτόματο,Ηλεκτρικό,448
1,2024-01-24 01:10:51.978821,https://www.instacar.gr/leasing/mercedes/eqa?s...,MERCEDES,EQA,1020,66.5,190,Αυτόματο,Ηλεκτρικό,0
2,2024-01-24 01:11:13.349917,https://www.instacar.gr/leasing/mercedes/cla?s...,MERCEDES,CLA,1043,1332,163,Αυτόματο,Βενζίνη,5.9
3,2024-01-24 01:12:00.977956,https://www.instacar.gr/leasing/tesla/model-3?...,TESLA,MODEL,1140,75,462,Αυτόματο,Ηλεκτρικό,580
4,2024-01-24 01:12:49.294781,https://www.instacar.gr/leasing/jaguar/e-pace?...,JAGUAR,E-PACE,1196,1997,200,Αυτόματο,Βενζίνη,8.9
5,2024-01-24 01:13:10.456081,https://www.instacar.gr/leasing/land-rover/ran...,LAND,ROVER,1197,1998,180,Αυτόματο,Πετρέλαιο,6
6,2024-01-24 01:13:35.060171,https://www.instacar.gr/leasing/volvo/xc60?sku...,VOLVO,XC60,1216,1969,197,Αυτόματο,Πετρέλαιο,6.4
7,2024-01-24 01:13:59.013543,https://www.instacar.gr/leasing/land-rover/ran...,LAND,ROVER,1218,1998,204,Αυτόματο,Πετρέλαιο,8.1
8,2024-01-24 01:14:34.442832,https://www.instacar.gr/leasing/tesla/model-3?...,TESLA,MODEL,1250,75,462,Αυτόματο,Ηλεκτρικό,580
9,2024-01-24 01:15:00.787051,https://www.instacar.gr/leasing/mercedes/glc?s...,MERCEDES,GLC,1265,1999,227,Αυτόματο,Βενζίνη,7.3


In [42]:
df_1_2 = pd.concat([df, df2], axis = 0, ignore_index=True)

In [43]:
df_1_2

,date,link,brand,model,price,displacement,horsepower,gearbox,fuel,consumption
0,2024-01-23 22:18:20.080421,https://www.instacar.gr/leasing/citroen/ami?sk...,CITROEN,AMI,230,5.5,10,Αυτόματο,Ηλεκτρικό,75
1,2024-01-23 22:18:40.544492,https://www.instacar.gr/leasing/carver/cargo?s...,CARVER,CARGO,273,5.4,10,Αυτόματο,Ηλεκτρικό,130
2,2024-01-23 22:18:59.857525,https://www.instacar.gr/leasing/seat/mii?skuId...,SEAT,Mii,285,999,60,Χειροκίνητο,Βενζίνη,4.5
3,2024-01-23 22:19:20.164083,https://www.instacar.gr/leasing/skoda/citigo?s...,SKODA,CITIGO,285,999,60,Χειροκίνητο,Βενζίνη,4.1
4,2024-01-23 22:19:39.028297,https://www.instacar.gr/leasing/fiat/panda?sku...,FIAT,PANDA,290,1242,69,Χειροκίνητο,Βενζίνη,5.3
...,...,...,...,...,...,...,...,...,...,...
388,2024-01-24 00:58:19.412556,https://www.instacar.gr/leasing/mercedes/gla?s...,MERCEDES,GLA,992,1332,163,Αυτόματο,Βενζίνη,7.3
389,2024-01-24 00:58:43.568194,https://www.instacar.gr/leasing/toyota/proace-...,TOYOTA,PROACE,993,1998,144,Χειροκίνητο,Πετρέλαιο,6.4
390,2024-01-24 00:59:07.982562,https://www.instacar.gr/leasing/mercedes/glc?s...,MERCEDES,GLC,999,1991,320,Αυτόματο,Βενζίνη,2.2
391,2024-01-24 00:59:32.528091,https://www.instacar.gr/leasing/volkswagen/id-...,VOLKSWAGEN,ID.4,1000,77,204,Αυτόματο,Ηλεκτρικό,520


In [44]:
instacar_df = pd.concat([df_1_2, df3], axis = 0, ignore_index=True)

In [45]:
instacar_df

,date,link,brand,model,price,displacement,horsepower,gearbox,fuel,consumption
0,2024-01-23 22:18:20.080421,https://www.instacar.gr/leasing/citroen/ami?sk...,CITROEN,AMI,230,5.5,10,Αυτόματο,Ηλεκτρικό,75
1,2024-01-23 22:18:40.544492,https://www.instacar.gr/leasing/carver/cargo?s...,CARVER,CARGO,273,5.4,10,Αυτόματο,Ηλεκτρικό,130
2,2024-01-23 22:18:59.857525,https://www.instacar.gr/leasing/seat/mii?skuId...,SEAT,Mii,285,999,60,Χειροκίνητο,Βενζίνη,4.5
3,2024-01-23 22:19:20.164083,https://www.instacar.gr/leasing/skoda/citigo?s...,SKODA,CITIGO,285,999,60,Χειροκίνητο,Βενζίνη,4.1
4,2024-01-23 22:19:39.028297,https://www.instacar.gr/leasing/fiat/panda?sku...,FIAT,PANDA,290,1242,69,Χειροκίνητο,Βενζίνη,5.3
...,...,...,...,...,...,...,...,...,...,...
429,2024-01-24 01:26:46.101318,https://www.instacar.gr/leasing/porsche/cayenn...,PORSCHE,CAYENNE,2550,2995,470,Αυτόματο,Βενζίνη,2.5
430,2024-01-24 01:27:14.385821,https://www.instacar.gr/leasing/land-rover/def...,LAND,ROVER,2613,2996,250,Αυτόματο,Πετρέλαιο,8.9
431,2024-01-24 01:27:34.780538,https://www.instacar.gr/leasing/porsche/macan?...,PORSCHE,MACAN,2630,2894,380,Αυτόματο,Βενζίνη,11.1
432,2024-01-24 01:27:57.691858,https://www.instacar.gr/leasing/land-rover/ran...,LAND,ROVER,2693,2997,300,Αυτόματο,Πετρέλαιο,7.4


In [46]:
import openpyxl

In [47]:
instacar_df.to_excel('Instacar_scraped_data.xlsx', index=False, sheet_name='Car_info')